In [2]:
# Multiple Participants -- Approach 4: Model Template
# Train & Test on Distinct Groups of Participants
# Approach 4: Use Small % of Test Participant's Data (Chronological Split)
# Iterate over each other participant & select most "similar" to other participants

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json
pd.options.mode.chained_assignment = None

In [34]:
# Key Features: op_num_people, of_pose_distance, of_gaze_distance, of_confidence, ros_mistakes_session, ros_ts_robot_talked, timestamp
use_key_features = True

# Leave One Out Run: test on each participant & train on remaining
leave_one_out_run = True

# Number of Estimators for Bagging
num_estimators = 10

# Sample Percentage for Bagging
sample_percentage = 0.75

# Threshold to Use
threshold_to_use = 0.75
use_all_if_lower_than_threshold = True

# Percentage of Test Participant Data to Use
test_percentage_to_use = 0.3

# File Name for Model Results
output_file = 'multi4_kf_xgb_results.csv'

metric_keys = ['accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1']

# 1 -- Load Data

In [5]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [6]:
data = [data5, data7, data9, data11, data12, data17, data18]
part = [5, 7, 9, 11, 12, 17, 18]

In [7]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

for i, d in enumerate(data):
    data[i] = d.drop(columns=remove)

In [8]:
# Remove rows where engagment NaN
for i,d in enumerate(data):
    data[i] = d[np.isfinite(d['engagement'])]
    
# Remove rows where engagment is -1
for i,d in enumerate(data):
    data[i] = d[d['engagement']>=0]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [9]:
# Main Columns
basic_cols = []
for i in data[0].columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [10]:
# Open Face Columns

of_cols = []
for i in data[0].columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [11]:
# Audio Columns

a_cols = []
for i in data[0].columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [12]:
# ROS Columns

ros_cols = []
for i in data[0].columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_games_session
ros_in_game
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [13]:
# For Window Only:
non_window_features = []
window_features = []
for i in data[0].columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


In [14]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

In [15]:
# Key Features (from feature analysis) 
# Note: Timestamp automatically included

key_features = [
'op_num_people',
'of_pose_distance',
'of_gaze_distance',
'of_confidence',
'ros_mistakes_session',
'ros_ts_robot_talked']

### Filter Feature Set (Optional)

In [16]:
# always include basic_cols, add desired group of features 
# features_to_keep = all_data.columns

if use_key_features:
    features_to_keep = basic_cols + key_features
else:
    features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features

for i,d in enumerate(data):
    data[i] = d[features_to_keep]

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [17]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [18]:
# Helper Function: Split a Participant's Data Chronologically 

def part_split(split, split_size):
    split = split.sort_values(['session_num', 'timestamp'], ascending=[True, True])
    bogus = split['engagement']

    split_train, split_test, bogus1, bogus2 = train_test_split(split, bogus, test_size=split_size, shuffle=False)
    return split_test, split_train

In [22]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing & Shuffle 

# train_part: an array of participants 
# test_part: an array of participants
# split_size: how much of each test participant's data to use for validation
def split_no_val(train_part, test_part):
    train_data = []
    for p in train_part:
        i = part.index(p)
        train_data.append(data[i].copy())
    
    test_data = []
    for p in test_part:
        i = part.index(p)
        test_data.append(data[i].copy())
        
    train_data = pd.concat(train_data, ignore_index=True, sort=True)
    test_data = pd.concat(test_data, ignore_index=True, sort=True)
    
    # Sort Training Data 
    train_data = train_data.sample(frac=1)
    train_data = train_data.sample(frac=1)
    train_data = train_data.sample(frac=1)

    X_train = train_data.drop(columns=['engagement', 'session_num', 'participant'])        
    X_test = test_data.drop(columns=['engagement', 'session_num', 'participant'])        

    y_train = train_data['engagement']
    y_test = test_data['engagement']

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in X_train.columns:
        mean = np.nanmean(X_train[c])
        std = np.nanstd(X_train[c])

        if std == 0:
            X_train[c] = (X_train[c]-mean)
            X_test[c] = (X_test[c]-mean)
        else:
            X_train[c] = (X_train[c]-mean)/(std)
            X_test[c] = (X_test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(X_train[c])

            X_train[c] = X_train[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
        else:
            max_val = np.nanmax(X_train[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 

    return X_train, y_train, X_test, y_test

In [23]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing & Shuffle 

# train_part: an array of participants 
# test_part: an array of participants
# split_size: how much of each test participant's data to use for validation
def split(train_part, test_part, split_size):
    train_data = []
    for p in train_part:
        i = part.index(p)
        train_data.append(data[i].copy())
    
    val_data = []
    test_data = []
    for p in test_part:
        i = part.index(p)
        if split_size > 0:
            p_test, p_train = part_split(data[i].copy(), split_size)
            val_data.append(p_train)
            test_data.append(p_test)
        else:
            test_data.append(data[i].copy())
        
    train_data = pd.concat(train_data, ignore_index=True, sort=True)
    val_data = pd.concat(val_data, ignore_index=True, sort=True)
    test_data = pd.concat(test_data, ignore_index=True, sort=True)
    
    # Sort Training & Validation Data 
    train_data = train_data.sample(frac=1)
    train_data = train_data.sample(frac=1)
    train_data = train_data.sample(frac=1)
    val_data = val_data.sample(frac=1)
    val_data = val_data.sample(frac=1)
    val_data = val_data.sample(frac=1)

    X_train = train_data.drop(columns=['engagement', 'session_num', 'participant'])        
    X_val = val_data.drop(columns=['engagement', 'session_num', 'participant'])  
    X_test = test_data.drop(columns=['engagement', 'session_num', 'participant'])        

    y_train = train_data['engagement']
    y_val = val_data['engagement']
    y_test = test_data['engagement']

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in X_train.columns:
        mean = np.nanmean(X_train[c])
        std = np.nanstd(X_train[c])

        if std == 0:
            X_train[c] = (X_train[c]-mean)
            X_val[c] = (X_val[c]-mean)
            X_test[c] = (X_test[c]-mean)
        else:
            X_train[c] = (X_train[c]-mean)/(std)
            X_val[c] = (X_val[c]-mean)/(std)
            X_test[c] = (X_test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(X_train[c])

            X_train[c] = X_train[c].fillna(min_val)
            X_val[c] = X_val[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
        else:
            max_val = np.nanmax(X_train[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_val[c] = X_val[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 

    return X_train, y_train, X_val, y_val, X_test, y_test

In [24]:
# MODEL HERE
# Multiple Participants Approach 4: 
# Approach 4: Use Small % of Test Participant's Data (Chronological Split)
# Iterate over each other participant & select most "similar" to test participant

# Inputs: X_train, y_train, X_val, y_val, X_test, y_test

# Output: Accuracy Metrics as Dictionary 

# IMPORTS HERE
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn import neighbors

def model(X_train, y_train, X_test, y_test, is_verbose, use_bagging):    
    # Model Here: must create pred & scores arrays
    if use_bagging:
        clf = BaggingClassifier(XGBClassifier(n_estimators=100), n_estimators=num_estimators, max_samples=sample_percentage)
        clf.fit(X_train.values, y_train.values)
        scores = clf.predict_proba(X_test.values)[:,1]
        pred = [round(value) for value in scores]   
    else:
        clf = XGBClassifier(n_estimators=100)
        clf.fit(X_train.values, y_train.values)
        scores = clf.predict(X_test.values)
        pred = [round(value) for value in scores] 

    # Evaluation
    results = {}
    accuracy = accuracy_score(y_test, pred)
    results['accuracy'] = accuracy
    
    if is_verbose:
        try:
            auc = roc_auc_score(y_test, scores)
        except:
            auc = np.nan
        results['auc'] = auc

        precision = metrics.precision_score(y_test, pred, average=None)
        recall = metrics.recall_score(y_test, pred, average=None)
        f1 = metrics.f1_score(y_test, pred, average=None)

        results['precision_0'], results['precision_1'] = precision[0], precision[1]
        results['recall_0'], results['recall_1'] = recall[0], recall[1]
        results['f1_0'], results['f1_1'] = f1[0], f1[1]
    
    return results

In [31]:
all_pair_val_results = {}
for test_part in part:
    all_pair_val_results[test_part] = {}
    for train_part in part:
        if (train_part == test_part):
            continue
            
        new_results = {}
        
        # Train - Test Split (and preprocessing)
        X_train, y_train, X_val, y_val, X_test, y_test = split([train_part], [test_part], test_percentage_to_use)
        
        X_train = X_train.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        X_val = X_val.reset_index(drop=True)
        y_val = y_val.reset_index(drop=True)
        
        # Evaluate on Val
        results = model(X_train, y_train, X_val, y_val, False, False)
        print('Train:', train_part, '\tTest:', test_part, results['accuracy'])
        all_pair_val_results[test_part][train_part] = results['accuracy']

Train: 7 	Test: 5 0.8165249088699879
Train: 9 	Test: 5 0.7002098751794985
Train: 11 	Test: 5 0.8074671379653153
Train: 12 	Test: 5 0.7501380757759859
Train: 17 	Test: 5 0.7218601568540816
Train: 18 	Test: 5 0.8269082072241246
Train: 5 	Test: 7 0.7624219193639977
Train: 9 	Test: 7 0.7472316865417377
Train: 11 	Test: 7 0.8099091425326519
Train: 12 	Test: 7 0.7932992617830777
Train: 17 	Test: 7 0.7314026121521863
Train: 18 	Test: 7 0.827725724020443
Train: 5 	Test: 9 0.7301728520589731
Train: 7 	Test: 9 0.7168912048805287
Train: 11 	Test: 9 0.748220640569395
Train: 12 	Test: 9 0.7092653787493645
Train: 17 	Test: 9 0.7204499237417387
Train: 18 	Test: 9 0.742691916624301
Train: 5 	Test: 11 0.755712866368604
Train: 7 	Test: 11 0.7370839542970691
Train: 9 	Test: 11 0.7039244908097367
Train: 12 	Test: 11 0.7189518132141083
Train: 17 	Test: 11 0.6043219076005961
Train: 18 	Test: 11 0.7768877297565823
Train: 5 	Test: 12 0.7926203991942867
Train: 7 	Test: 12 0.8023255813953488
Train: 9 	Test: 12 

In [26]:
all_pair_test_results = {}
for test_part in part:
    all_pair_test_results[test_part] = {}
    for train_part in part:
        if (train_part == test_part):
            continue
            
        new_results = {}
        
        # Train - Test Split (and preprocessing)
        X_train, y_train, X_test, y_test = split_no_val([train_part], [test_part])
        
        X_train = X_train.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        
        # Evaluate on Val
        results = model(X_train, y_train, X_test, y_test, False, False)
        print('Train:', train_part, '\tTest:', test_part, results['accuracy'])
        all_pair_test_results[test_part][train_part] = results['accuracy']

Train: 7 	Test: 5 0.8042217582927395
Train: 9 	Test: 5 0.6828268769813655
Train: 11 	Test: 5 0.8104074847289878
Train: 12 	Test: 5 0.7744529498182943
Train: 17 	Test: 5 0.706873888502281
Train: 18 	Test: 5 0.81643856800433
Train: 5 	Test: 7 0.7256782271688363
Train: 9 	Test: 7 0.7685580840703568
Train: 11 	Test: 7 0.8235118751863262
Train: 12 	Test: 7 0.8050780085461592
Train: 17 	Test: 7 0.7481864255192289
Train: 18 	Test: 7 0.8256981019576667
Train: 5 	Test: 9 0.7470753080378987
Train: 7 	Test: 9 0.7279035630087629
Train: 11 	Test: 9 0.7754548285218629
Train: 12 	Test: 9 0.72821493705796
Train: 17 	Test: 9 0.735020684133268
Train: 18 	Test: 9 0.7574840976824875
Train: 5 	Test: 11 0.7569435389229365
Train: 7 	Test: 11 0.7349067675055418
Train: 9 	Test: 11 0.6783587603772765
Train: 12 	Test: 11 0.7186943104272613
Train: 17 	Test: 11 0.6089016386317208
Train: 18 	Test: 11 0.7559438431781632
Train: 5 	Test: 12 0.7782477728641928
Train: 7 	Test: 12 0.810869704543998
Train: 9 	Test: 12 0.7

In [29]:
for p in part:
    avg = []
    for test, other in all_pair_test_results.items():
        if (p != test):
            avg.append(other[p])
    print(p, np.mean(avg))

5 0.7474484306574168
7 0.776552143653793
9 0.7294597464129841
11 0.7978297312588699
12 0.758049998715517
17 0.7079937988653381
18 0.7825605007961296


In [35]:
all_results = []
for n in range(1, len(part)):    
    if leave_one_out_run:
        if n != len(part)-1:
            continue
    
    comb = combinations(part, n)
    print()
    
    for c in comb:
        print()
        new_results = {}
        for m in metric_keys:
            new_results[m] = []
        
        # Determine Train - Test Participants
        train_part_candidate = [x for x in c]
        test_part = [p for p in part if p not in c]
        print('Train:', train_part_candidate, '\tTest:', test_part)
        
        # Evaluate each test participant separately
        for test in test_part:
            # Lookup best train participants to use for this test participant
            train_part = []
            best_part = -1
            best_part_acc = -1
            for candidate in train_part_candidate:
                lookup_acc_val = all_pair_val_results[test][candidate]
                
                lookup_acc_gen = []
                for other in train_part_candidate:
                    if (candidate == other):
                        continue
                    lookup_acc_gen.append(all_pair_test_results[other][candidate])
                lookup_acc_gen = np.mean(lookup_acc_gen)
                
                lookup_acc = (lookup_acc_val + lookup_acc_gen)/2
                
                if (lookup_acc>threshold_to_use):
                    train_part.append(candidate)
                if (lookup_acc>best_part_acc):
                    best_part = candidate
                    best_part_acc = lookup_acc
            if len(train_part)==0:
                if use_all_if_lower_than_threshold:
                    train_part = train_part_candidate
                else:
                    train_part.append(best_part)
            print('For ', test, 'using train participants: ', train_part)
            
            # Train - Test Split (and preprocessing)
            X_train, y_train, X_val, y_val, X_test, y_test = split(train_part, [test], test_percentage_to_use)
            X_train = X_train.append(X_val)
            y_train = y_train.append(y_val)
            
            idx = np.random.permutation(len(X_train))
            X_train = X_train.iloc[idx]
            y_train = y_train.iloc[idx]
            X_train = X_train.reset_index(drop=True)
            y_train = y_train.reset_index(drop=True)
        
            # Evaluate for this test participant  
            results = model(X_train, y_train, X_test, y_test, True, True)
            print(results)
            for m in metric_keys:
                new_results[m].append(results[m])
        
        # Combine results across test participants
        final_results = {}
        for m in metric_keys:
            final_results[m] = np.mean(new_results[m])
        final_results['train_participants'] = ", ".join(str(x) for x in train_part_candidate)
        final_results['test_participants'] = ", ".join(str(x) for x in test_part)
        all_results.append(final_results)



Train: [5, 7, 9, 11, 12, 17] 	Test: [18]
For  18 using train participants:  [11]
{'f1_0': 0.17708333333333334, 'recall_0': 0.10365853658536585, 'precision_0': 0.6071428571428571, 'auc': 0.741333992634552, 'f1_1': 0.851255230125523, 'accuracy': 0.7480510276399717, 'recall_1': 0.9762476007677543, 'precision_1': 0.7546364985163204}

Train: [5, 7, 9, 11, 12, 18] 	Test: [17]
For  17 using train participants:  [5, 7, 9, 11, 12, 18]
{'f1_0': 0.6250445950767035, 'recall_0': 0.4735135135135135, 'precision_0': 0.919202518363064, 'auc': 0.8859273255017936, 'f1_1': 0.7830753353973168, 'accuracy': 0.7251569037656904, 'recall_1': 0.9609929078014184, 'precision_1': 0.6607453848833159}

Train: [5, 7, 9, 11, 17, 18] 	Test: [12]
For  12 using train participants:  [11]
{'f1_0': 0.8175771971496436, 'recall_0': 0.7459904638058084, 'precision_0': 0.9043615344193379, 'auc': 0.8994131263810482, 'f1_1': 0.8509316770186336, 'accuracy': 0.8359324930570391, 'recall_1': 0.9233361415332771, 'precision_1': 0.78905

In [ ]:
end_results = pd.DataFrame(columns=['train_participants', 'test_participants', 'accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1'])
end_results = end_results.append(all_results, ignore_index=True, sort=True)

In [ ]:
end_results.head()

In [ ]:
end_results.to_csv(output_file, index=False)